***
# Car prices
***

1. Répertorier l'ensemble des marques de véhicules
2. Répertorier l'ensemble des modèles de véhicules
3. Répertorier l'ensemble des prix des véhicules
***

In [1]:
# Create folders and getting path
import os
cwd = os.getcwd()

# Loop to create folders
folder_names = ['Dataframe', 'Output', 'Data']

folders = {}
for folder_name in folder_names:
    folders[folder_name] = os.path.join(cwd, folder_name)

    if not os.path.exists(os.path.join(cwd, folder_name)):
        os.makedirs(os.path.join(cwd, folder_name))
        print(f'Le dossier « {folder_name} » a été créé')

    else:
        print(f'Le dossier « {folder_name} » est existant')


# Création des variables de path

# Déterminer si os est win ou linux pour définir les path
if os.name == 'nt':
    slash = '\\'
elif os.name == 'posix':
    slash = '/'

path_prog =     cwd + slash
path_data =     folders['Data'] + slash
path_df =       folders['Dataframe'] + slash
path_output =   folders['Output'] + slash

# Mettre \\ pour éviter les erreurs
path_dict = [path_prog, path_data, path_df, path_output]
for path in path_dict:
    path = path.replace('\\','\\\\')


# Détermination de l'année
print()
print(path_prog)
print(path_data)
print(path_df)
print(path_output)

Le dossier « Dataframe » est existant
Le dossier « Output » est existant
Le dossier « Data » est existant

c:\Users\segch001\Documents\GitHub\car_sales_predictions\
c:\Users\segch001\Documents\GitHub\car_sales_predictions\Data\
c:\Users\segch001\Documents\GitHub\car_sales_predictions\Dataframe\
c:\Users\segch001\Documents\GitHub\car_sales_predictions\Output\


***

In [3]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup

pd.set_option('display.max_colwidth', 1000) 

link_original        = 'https://www.guideautoweb.com'
link_constructeur   = 'https://www.guideautoweb.com/constructeurs/'
response_api = requests.get(link_constructeur, verify=False)
print(response_api.status_code)     # 200 is ok

200


c:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [4]:
# Get the data from api
data = response_api.text
soup = BeautifulSoup(data, 'html.parser')

In [6]:
# Extraire les marques de la page principale
ul_element = soup.find('ul', id='brands-index-list')

data = {
    'marque' : [],
    'link_model' : []
}

# Extract information from <a> elements within the <ul> element
for a_element in ul_element.find_all('a'):
    a_text = a_element.get_text().strip()
    a_href = a_element.get('href')
    link_model = link_original + a_href

    data['marque'].append(a_text)
    data['link_model'].append(link_model)

df = pd.DataFrame(data)
df.head(3)

,link_model,marque
0,https://www.guideautoweb.com/constructeurs/acura/,Acura
1,https://www.guideautoweb.com/constructeurs/alf...,Alfa Romeo
2,https://www.guideautoweb.com/constructeurs/all...,Allard


In [37]:
# Loop pour Extraire modèle de la seconde page
count = 0

data = {
    'model' : [],
    'link' : [],
    'production' : []
    }

for link in df['link_model']:
    if count >= 2:
        break  # Exit the loop after the second iteration

    model_api = requests.get(link, verify=False)
    count += 1


    model_data = model_api.text
    model_soup = BeautifulSoup(model_data, 'html.parser')


    # Extract model en production
    section_production = model_soup.find_all('div', class_='s')


    extracted_text_list = []
    extracted_ref_list = []

    for section in section_production:
        try:
            a_elements = section.find_all('a', class_='e-a e-t')

            for a_element in a_elements:
                extracted_text_list.append(a_element.get_text())
                extracted_ref_list.append(a_element.get('href'))

        except AttributeError:
            continue

    for text, ref in zip(extracted_text_list, extracted_ref_list):
        data['model'].append(text)
        data['link'].append(link_original + ref)
        data['production'].append(True)

    # Extracting model non en production
    section_production_autre = model_soup.find('ul', class_='eg eg-t1 eg-sz-s')
    if section_production_autre:
        for a_element in section_production_autre.find_all('a', class_='txt'):
            try:
                text = a_element.get_text()
                data['model'].append(text)
                data['link'].append(link[:-1] + a_element.get('href'))
                data['production'].append(False)
                
            except AttributeError:
                continue

    df_model = pd.DataFrame(data)
    
df_model.head(3)     

c:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
c:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


,link,model,production
0,https://www.guideautoweb.com/constructeurs/acura/integra/2023/,Acura Integra,True
1,https://www.guideautoweb.com/constructeurs/acura/mdx/2023/,Acura MDX,True
2,https://www.guideautoweb.com/constructeurs/acura/rdx/2023/,Acura RDX,True


In [20]:
df_model.to_excel(path_output + 'df_model.xlsx')

In [78]:
# Extraire la liste des années modèle disponibles

# 1. Extraire le code HTML de la page
acura_integra_test_link = 'https://www.guideautoweb.com/constructeurs/acura/integra/2023/'
acura_rdx_test_link = 'https://www.guideautoweb.com/constructeurs/acura/rdx/2023/'



anne_mod_api = requests.get(acura_rdx_test_link, verify=False)
anne_mod_soup = BeautifulSoup(anne_mod_api.text, 'html.parser')
anne_mod_soup

annee_list = []

# 2. Isoler les parties du code pertinantes
h1_section = anne_mod_soup.find('h1', class_='st st-s3')

for option in h1_section.find_all('option'):
    annee_list.append(link_original + option.get('value'))

# data_annee_mod['annee_list'] = annee_list

data_annee_mod = {
    'link':         [acura_rdx_test_link],
    'annee_list':   annee_list
}
data_annee_mod

c:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'annee_list': ['https://www.guideautoweb.com/constructeurs/acura/rdx/2023/',
  'https://www.guideautoweb.com/constructeurs/acura/rdx/2022/',
  'https://www.guideautoweb.com/constructeurs/acura/rdx/2021/',
  'https://www.guideautoweb.com/constructeurs/acura/rdx/2020/',
  'https://www.guideautoweb.com/constructeurs/acura/rdx/2019/',
  'https://www.guideautoweb.com/constructeurs/acura/rdx/2018/',
  'https://www.guideautoweb.com/constructeurs/acura/rdx/2017/',
  'https://www.guideautoweb.com/constructeurs/acura/rdx/2016/',
  'https://www.guideautoweb.com/constructeurs/acura/rdx/2015/',
  'https://www.guideautoweb.com/constructeurs/acura/rdx/2014/',
  'https://www.guideautoweb.com/constructeurs/acura/rdx/2013/',
  'https://www.guideautoweb.com/constructeurs/acura/rdx/2012/',
  'https://www.guideautoweb.com/constructeurs/acura/rdx/2011/',
  'https://www.guideautoweb.com/constructeurs/acura/rdx/2010/',
  'https://www.guideautoweb.com/constructeurs/acura/rdx/2009/',
  'https://www.guideautowe

In [80]:
df_test = pd.DataFrame(data_annee_mod)
annee_list = df_test['annee_list'].tolist()
annee_list

ValueError: arrays must all be same length

In [72]:
import pandas as pd

data = {
    'link': ['https://www.guideautoweb.com/constructeurs/acura/rdx/2023/'],
    'annee_list': [
        'https://www.guideautoweb.com/constructeurs/acura/rdx/2023/',
        'https://www.guideautoweb.com/constructeurs/acura/rdx/2022/',
        'https://www.guideautoweb.com/constructeurs/acura/rdx/2021/',
        'https://www.guideautoweb.com/constructeurs/acura/rdx/2020/',
        'https://www.guideautoweb.com/constructeurs/acura/rdx/2019/',
        'https://www.guideautoweb.com/constructeurs/acura/rdx/2018/',
        'https://www.guideautoweb.com/constructeurs/acura/rdx/2017/',
        'https://www.guideautoweb.com/constructeurs/acura/rdx/2016/',
        'https://www.guideautoweb.com/constructeurs/acura/rdx/2015/',
        'https://www.guideautoweb.com/constructeurs/acura/rdx/2014/',
        'https://www.guideautoweb.com/constructeurs/acura/rdx/2013/',
        'https://www.guideautoweb.com/constructeurs/acura/rdx/2012/',
        'https://www.guideautoweb.com/constructeurs/acura/rdx/2011/',
        'https://www.guideautoweb.com/constructeurs/acura/rdx/2010/',
        'https://www.guideautoweb.com/constructeurs/acura/rdx/2009/',
        'https://www.guideautoweb.com/constructeurs/acura/rdx/2008/'
    ]
}

df = pd.DataFrame(data)
df

ValueError: Shape of passed values is (2, 16), indices imply (2, 1)

In [54]:
# Ajouter la liste des années modèle au df des 


df_model

,link,model,production
0,https://www.guideautoweb.com/constructeurs/acura/integra/2023/,Acura Integra,True
1,https://www.guideautoweb.com/constructeurs/acura/mdx/2023/,Acura MDX,True
2,https://www.guideautoweb.com/constructeurs/acura/rdx/2023/,Acura RDX,True
3,https://www.guideautoweb.com/constructeurs/acura/tlx/2023/,Acura TLX,True
4,https://www.guideautoweb.com/constructeurs/acura/constructeurs/acura/cl/,Acura CL,False
5,https://www.guideautoweb.com/constructeurs/acura/constructeurs/acura/concept/,Acura Concept,False
6,https://www.guideautoweb.com/constructeurs/acura/constructeurs/acura/csx/,Acura CSX,False
7,https://www.guideautoweb.com/constructeurs/acura/constructeurs/acura/el/,Acura EL,False
8,https://www.guideautoweb.com/constructeurs/acura/constructeurs/acura/ilx/,Acura ILX,False
9,https://www.guideautoweb.com/constructeurs/acura/constructeurs/acura/nsx/,Acura NSX,False


In [ ]:
# Extract list of all models
# Extract list of web links to the models

In [ ]:
# Extract list of 'model en production'
# Extract web link to those models

In [ ]:
# Extract price and fuel consomption for every model